In [1]:
import torch
import os
import sys
import importlib.util

from torch import Tensor
import nerfacc

from nerfacc.estimators.prop_net import (
    PropNetEstimator,
    get_proposal_requires_grad_fn,
)


/home/ccl/anaconda3/envs/NeRF/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
exp_dir = "output/test2"

# Create a module spec
spec = importlib.util.spec_from_file_location('ngp_appearance', f'{exp_dir}/ngp_appearance.py')
ngp_appearance = importlib.util.module_from_spec(spec)
spec.loader.exec_module(ngp_appearance)
NGPDensityField = ngp_appearance.NGPDensityField
NGPRadianceField = ngp_appearance.NGPRadianceField


device = "cuda:0"
# scene parameters
unbounded = True
aabb = torch.tensor([-1.0, -1.0, -1.0, 1.0, 1.0, 1.0], device=device)
near_plane = 0.2  # TODO: Try 0.02
far_plane = 1e3
# dataset parameters
train_dataset_kwargs = {"color_bkgd_aug": "random", "factor": 2}
test_dataset_kwargs = {"factor": 4}
# model parameters
proposal_networks = [
    NGPDensityField(aabb=aabb).to(device),
    NGPDensityField(aabb=aabb).to(device),
]

estimator = PropNetEstimator().to(device)
radiance_field = NGPRadianceField(aabb=aabb, unbounded=unbounded, max_resolution=4096*2, n_levels=16, log2_hashmap_size=17).to(device)

In [9]:
radiance_field.load_state_dict(torch.load(os.path.join(exp_dir, 'radiance_field.pth')))
estimator.load_state_dict(torch.load(os.path.join(exp_dir, 'estimator.pth')))

for i, net in enumerate(proposal_networks):
    net.load_state_dict(torch.load(os.path.join(exp_dir, f'proposal_network_{i}.pth')), strict=False)

In [ ]:

from utils import (
    render_image_with_propnet,
)

rgb, acc, depth, _, = render_image_with_propnet(
    radiance_field,
    proposal_networks,
    estimator,
    rays,
    # rendering options
    num_samples=num_samples,
    num_samples_per_prop=num_samples_per_prop,
    near_plane=near_plane,
    far_plane=far_plane,
    sampling_type=sampling_type,
    opaque_bkgd=opaque_bkgd,
    render_bkgd=render_bkgd,
    # test options
    test_chunk_size=args.test_chunk_size,
    img=img,
)